In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_column',None)
pd.set_option('display.max_row',5000)

In [3]:
records = pd.read_csv("/Users/mk2/Documents - Gabriel’s MacBook Pro/dadaset/records.csv")

### Preprocess

In [4]:
records = records.drop(columns= ["Unnamed: 0"])
records.infer_objects()
records["date"] = pd.to_datetime(records["date"],format='%d/%m/%y')

In [5]:
records = records.sort_values(by=["date","race_index","result"], ascending=False)
records.reset_index(inplace=True, drop=True)

In [6]:
records.columns

Index(['race_id', 'race_index', 'result', 'date', 'venue', 'dist',
       'turf_quality', 'race_class', 'draw', 'rating', 'trainer', 'jockey',
       'lbw', 'win_odds', 'actual_weight', 'running_pos1', 'running_pos2',
       'running_pos3', 'running_pos4', 'running_pos5', 'finish_time',
       'declared_weight', 'gear', 'id', 'name', 'details', 'country_of_origin',
       'age', 'colour', 'sex', 'import_type', 'season_stakes', 'total_stakes',
       'win_counts', 'second_count', 'third_count', 'horse_exp',
       'no_starts_past_10', 'current_stable_loc', 'arrival_date', 'trainer.1',
       'owner', 'current_rating', 'start_ofseason_rating', 'sire', 'dam',
       'dams_sire', 'same_sire'],
      dtype='object')

In [7]:
records.drop(columns=['season_stakes', 'total_stakes',
       'win_counts', 'second_count', 'third_count', 'horse_exp',
       'no_starts_past_10', 'current_stable_loc', 'arrival_date',
       'current_rating', 'start_ofseason_rating'],inplace=True)

### Variables

#### Age

In [8]:
records.loc[:,('rest_time')] = (records.groupby('name')["date"].diff(periods=-1).dt.days.fillna(0))

records.loc[:,('cumul_rest_time')] = records.groupby('name')['rest_time'].cumsum().fillna(0)

records.loc[:,('real_age')] = (records['age'] - records['cumul_rest_time']/365).round()

#### General

In [9]:
records.loc[:,('speed')] = (records['dist'] / records['finish_time']).round(2)

records.loc[:,('horse_exp')] = records.groupby(['name']).cumcount(ascending=False)

records.loc[:,("distance_exp")] = records.groupby(["name","dist"]).cumcount(ascending=False)

records.loc[:,("horse_jockey_exp")] = records.groupby(["name","jockey"]).cumcount(ascending=False)

#### Prev

In [10]:
records.loc[:,('prev_speed')] = records.groupby('name')[('speed')].shift(periods=-1)

records.loc[:,('prev_result')] = records.groupby('name')[('result')].shift(periods=-1)

records["prev_actual_weight"] = records.groupby('name')['actual_weight'].shift(periods=-1)

records["prev_declared_weight"] = records.groupby(['name'])['declared_weight'].shift(periods=-1)

records.loc[:,('prev_time')] = records.groupby(["name","dist"])[('finish_time')].shift(periods=-1)

records.loc[:,('prev_running_pos4')] = records.groupby('name')[('running_pos4')].shift(periods=-1)

records.loc[:,('prev_running_pos3')] = records.groupby('name')[('running_pos3')].shift(periods=-1)

records.loc[:,('prev_running_pos2')] = records.groupby('name')[('running_pos2')].shift(periods=-1)

records.loc[:,('prev_running_pos1')] = records.groupby('name')[('running_pos1')].shift(periods=-1)

#### Change

In [11]:
records["actual_wt_change"] = records['actual_weight'] - records['prev_actual_weight']

records["declared_wt_change"] = records['declared_weight'] - records['prev_declared_weight']

records["prev_speed_change"] = records['prev_speed'] - records['prev_speed'].shift(periods=-1)

#### Ranks

In [12]:
records.loc[:,("odds_rank")] = records.loc[:,("win_odds")].rank(pct=True, ascending=False)

records.loc[:,("real_age_rank")] = records.groupby(["race_index","date"])["real_age"].rank(pct=True, ascending=False)

records.loc[:,("cond_odds_rank")] = records.groupby(["race_index","date"])["win_odds"].rank(pct=True, ascending=False)

records.loc[:,("prev_speed_rank")] = records.groupby(["race_index","date"])["prev_speed"].rank(pct=True, ascending=True)

records.loc[:,("distance_exp_rank")] = records.groupby(["race_index","date"])["distance_exp"].rank(pct=True, ascending=True)

records.loc[:,("actual_weight_rank")] = records.groupby(["race_index","date"])["actual_weight"].rank(pct=True, ascending=True)

records.loc[:,("declared_weight_rank")] = records.groupby(["race_index","date"])["declared_weight"].rank(pct=True, ascending=True)

records.loc[:,("actual_wt_change_rank")] = records.groupby(["race_index","date"])["actual_wt_change"].rank(pct=True, ascending=True)

records.loc[:,('prev_running_pos4_rank')] = records.groupby(["race_index","date"])["prev_running_pos4"].rank(pct=True, ascending=True)

records.loc[:,('prev_running_pos3_rank')] = records.groupby(["race_index","date"])["prev_running_pos3"].rank(pct=True, ascending=True)

records.loc[:,('prev_running_pos2_rank')] = records.groupby(["race_index","date"])["prev_running_pos2"].rank(pct=True, ascending=True)

records.loc[:,('prev_running_pos1_rank')] = records.groupby(["race_index","date"])["prev_running_pos1"].rank(pct=True, ascending=True)

In [13]:
records.groupby(["name"])

In [14]:
records.corrwith(records["real_age"])#.plot(kind='barh',vfigsize=(10,20),fontsize=12, grid = True)

race_id                  -0.168685
race_index               -0.168685
result                   -0.001876
dist                      0.274297
race_class               -0.032953
draw                     -0.020960
rating                    0.042691
win_odds                 -0.096879
actual_weight            -0.020577
running_pos1              0.051540
running_pos2              0.051706
running_pos3              0.038247
running_pos4             -0.011866
running_pos5             -0.013447
finish_time               0.274027
declared_weight           0.050152
age                       0.618917
rest_time                -0.050991
cumul_rest_time          -0.268829
real_age                  1.000000
speed                    -0.226136
horse_exp                 0.838517
distance_exp              0.590920
horse_jockey_exp          0.300495
prev_speed               -0.201046
prev_result              -0.006597
prev_actual_weight       -0.027987
prev_declared_weight      0.043134
prev_time           

#### Rolling

In [46]:
roll_records = records.copy()

roll_records = roll_records.set_index(['date'], drop=False)

roll_records.sort_index(ascending=True,inplace=True)

In [47]:
#result_mean

roll_records["168D_average"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("168D", min_periods=1, closed="neither").mean())
roll_records["168D_average_rank"] = roll_records.groupby(["race_index"])["168D_average"].rank(pct=True, ascending=True)

roll_records["112D_average"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("112D", min_periods=1, closed="neither").mean())
roll_records["112D_average_rank"] = roll_records.groupby(["race_index"])["112D_average"].rank(pct=True, ascending=True)

roll_records["84D_average"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("84D", min_periods=1, closed="neither").mean())
roll_records["84D_average_rank"] = roll_records.groupby(["race_index"])["84D_average"].rank(pct=True, ascending=True)

#result_median

roll_records["168D_median"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("168D", min_periods=1, closed="neither").median())
roll_records["168D_median_rank"] = roll_records.groupby(["race_index"])["168D_median"].rank(pct=True, ascending=True)

roll_records["112D_median"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("112D", min_periods=1, closed="neither").median())
roll_records["112D_median_rank"] = roll_records.groupby(["race_index"])["112D_median"].rank(pct=True, ascending=True)

roll_records["84D_median"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("84D", min_periods=1, closed="neither").median())
roll_records["84D_median_rank"] = roll_records.groupby(["race_index"])["84D_median"].rank(pct=True, ascending=True)

#result_max

roll_records["168D_worst"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("168D", min_periods=1, closed="neither").max())
roll_records["168D_worst_rank"] = roll_records.groupby(["race_index"])["168D_worst"].rank(pct=True, ascending=True)

roll_records["112D_worst"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("112D", min_periods=1, closed="neither").max())
roll_records["112D_worst_rank"] = roll_records.groupby(["race_index"])["112D_worst"].rank(pct=True, ascending=True)

roll_records["84D_worst"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("84D", min_periods=1, closed="neither").max())
roll_records["84D_worst_rank"] = roll_records.groupby(["race_index"])["84D_worst"].rank(pct=True, ascending=True)

#result_min

roll_records["168D_best"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("168D", min_periods=1, closed="neither").min())
roll_records["168D_best_rank"] = roll_records.groupby(["race_index"])["168D_best"].rank(pct=True, ascending=True)

roll_records["112D_best"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("112D", min_periods=1, closed="neither").min())
roll_records["112D_best_rank"] = roll_records.groupby(["race_index"])["112D_best"].rank(pct=True, ascending=True)

roll_records["84D_best"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("84D", min_periods=1, closed="neither").min())
roll_records["84D_best_rank"] = roll_records.groupby(["race_index"])["84D_best"].rank(pct=True, ascending=True)

In [48]:
#speed_mean

roll_records["168D_speed"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").mean())
roll_records["168D_speed_rank"] = roll_records.groupby(["race_index"])["168D_speed"].rank(pct=True, ascending=True)

roll_records["112D_speed"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").mean())
roll_records["112D_speed_rank"] = roll_records.groupby(["race_index"])["112D_speed"].rank(pct=True, ascending=True)

roll_records["84D_speed"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").mean())
roll_records["84D_speed_rank"] = roll_records.groupby(["race_index"])["84D_speed"].rank(pct=True, ascending=True)

#speed_median

roll_records["168D_speed_median"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").median())
roll_records["168D_speed_median_rank"] = roll_records.groupby(["race_index"])["168D_speed_median"].rank(pct=True, ascending=True)

roll_records["112D_speed_median"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").median())
roll_records["112D_speed_median_rank"] = roll_records.groupby(["race_index"])["112D_speed_median"].rank(pct=True, ascending=True)

roll_records["84D_speed_median"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").median())
roll_records["84D_speed_median_rank"] = roll_records.groupby(["race_index"])["84D_speed_median"].rank(pct=True, ascending=True)

#speed_max

roll_records["168D_speed_max"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").max())
roll_records["168D_speed_max_rank"] = roll_records.groupby(["race_index"])["168D_speed_max"].rank(pct=True, ascending=True)

roll_records["112D_speed_max"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").max())
roll_records["112D_speed_max_rank"] = roll_records.groupby(["race_index"])["112D_speed_max"].rank(pct=True, ascending=True)

roll_records["84D_speed_max"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").max())
roll_records["84D_speed_max_rank"] = roll_records.groupby(["race_index"])["84D_speed_max"].rank(pct=True, ascending=True)

#speed_min

roll_records["168D_speed_min"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").min())
roll_records["168D_speed_min_rank"] = roll_records.groupby(["race_index"])["168D_speed_min"].rank(pct=True, ascending=True)

roll_records["112D_speed_min"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").min())
roll_records["112D_speed_min_rank"] = roll_records.groupby(["race_index"])["112D_speed_min"].rank(pct=True, ascending=True)

roll_records["84D_speed_min"] = roll_records.groupby(["name"])["speed"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").min())
roll_records["84D_speed_min_rank"] = roll_records.groupby(["race_index"])["84D_speed_min"].rank(pct=True, ascending=True)

In [49]:
#prev_finish_time_mean

roll_records["168D_finish_time"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").mean())
roll_records["168D_finish_time_rank"] = roll_records.groupby(["race_index"])["168D_finish_time"].rank(pct=True, ascending=True)

roll_records["112D_finish_time"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").mean())
roll_records["112D_finish_time_rank"] = roll_records.groupby(["race_index"])["112D_finish_time"].rank(pct=True, ascending=True)

roll_records["84D_finish_time"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").mean())
roll_records["84D_finish_time_rank"] = roll_records.groupby(["race_index"])["84D_finish_time"].rank(pct=True, ascending=True)

#prev_finish_time_median

roll_records["168D_finish_time_median"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").median())
roll_records["168D_finish_time_median_rank"] = roll_records.groupby(["race_index"])["168D_finish_time_median"].rank(pct=True, ascending=True)

roll_records["112D_finish_time_median"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").median())
roll_records["112D_finish_time_median_rank"] = roll_records.groupby(["race_index"])["112D_finish_time_median"].rank(pct=True, ascending=True)

roll_records["84D_finish_time_median"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").median())
roll_records["84D_finish_time_median_rank"] = roll_records.groupby(["race_index"])["84D_finish_time_median"].rank(pct=True, ascending=True)

#prev_finish_time_min

roll_records["168D_finish_time_min"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").min())
roll_records["168D_finish_time_min_rank"] = roll_records.groupby(["race_index"])["168D_finish_time_min"].rank(pct=True, ascending=True)

roll_records["112D_finish_time_min"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").min())
roll_records["112D_finish_time_min_rank"] = roll_records.groupby(["race_index"])["112D_finish_time_min"].rank(pct=True, ascending=True)

roll_records["84D_finish_time_min"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").min())
roll_records["84D_finish_time_min_rank"] = roll_records.groupby(["race_index"])["84D_finish_time_min"].rank(pct=True, ascending=True)

#prev_finish_time_max

roll_records["168D_finish_time_max"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("168D", min_periods=1,closed="neither").max())
roll_records["168D_finish_time_max_rank"] = roll_records.groupby(["race_index"])["168D_finish_time_max"].rank(pct=True, ascending=True)

roll_records["112D_finish_time_max"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").max())
roll_records["112D_finish_time_max_rank"] = roll_records.groupby(["race_index"])["112D_finish_time_max"].rank(pct=True, ascending=True)

roll_records["84D_finish_time_max"] = roll_records.groupby(["name","dist"])["finish_time"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").max())
roll_records["84D_finish_time_max_rank"] = roll_records.groupby(["race_index"])["84D_finish_time_max"].rank(pct=True, ascending=True)

In [36]:
roll_records_multindex = roll_records.set_index(['date','race_index'], drop=False)

#prev_jockey_result_mean

roll_records_multindex["168D_jockey"] = roll_records_multindex.groupby(["name"])["prev_result"].transform(lambda s: s.rolling(6, min_periods=1).mean())
roll_records_multindex["168D_jockey_rank"] = roll_records_multindex.groupby(["race_id"])["168D_jockey"].rank(pct=True, ascending=True)

#roll_records["112D_jockey"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("112D", min_periods=1,closed="neither").mean())
#roll_records["112D_jockey_rank"] = roll_records.groupby(["race_index"])["112D_jockey"].rank(pct=True, ascending=True)

#roll_records["84D_jockey"] = roll_records.groupby(["name"])["result"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").mean())
#roll_records["84D_jockey_rank"] = roll_records.groupby(["race_index"])["84D_jockey"].rank(pct=True, ascending=True)

In [44]:
roll_records.groupby(["jockey"])["finish_time"].transform(lambda s: s.rolling("84D", min_periods=1,closed="neither").cumcount().mean())

ValueError: window must be an integer

In [20]:
records["jockey"].isna().value_counts()

False    16432
True         5
Name: jockey, dtype: int64

In [34]:
roll_records.corrwith(roll_records["result"])#.plot(kind='barh',figsize=(10,20),fontsize=12, grid = True)

race_id                         0.008526
race_index                      0.008526
result                          1.000000
dist                            0.005744
race_class                     -0.001794
draw                            0.134974
rating                         -0.039867
win_odds                        0.402648
actual_weight                  -0.076979
running_pos1                    0.187518
running_pos2                    0.238052
running_pos3                    0.602781
running_pos4                    0.864890
running_pos5                    0.938434
finish_time                     0.033994
declared_weight                -0.059077
age                            -0.019834
rest_time                       0.003478
cumul_rest_time                -0.024625
real_age                       -0.001876
speed                          -0.268330
horse_exp                      -0.045238
distance_exp                   -0.071984
horse_jockey_exp               -0.091361
prev_speed      

## race_checker

In [23]:
records_notna = records[records.result.notna()]
participants_count = records_notna.groupby(['race_index','date'])['id'].count()
result_sum = records_notna.groupby(['race_index','date'])['result'].sum()

race_checker = (lambda p,r: r == (p / 2)*(1 + p))
race_checked = race_checker(participants_count, result_sum)
race_checked.value_counts()

False    2811
True      322
dtype: int64

In [41]:
roll_records["result"].isna()

date        race_index
2022-02-12  415.0         False
            415.0         False
            415.0         False
            415.0         False
            415.0         False
            415.0         False
            415.0         False
            420.0         False
            417.0         False
            417.0         False
            415.0         False
            417.0         False
            417.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0         False
            419.0          True
            418.0         False
            419.0         False
            418.0         False
            418.0         False
            417.0         False
            417.0

In [25]:

def race_checker(p, r):
    
    participants = records.groupby(['race_index','date'])['id'].count()
    result = records.groupby(['race_index','date'])['result'].sum()
    
    for x in p :
        
        if x == 14 and y == 105:
            return True       
        else:
            return False
